<a href="https://colab.research.google.com/github/orel33/bloc3/blob/master/traceroute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traceroute

## Intro Musicale (ou pas)

https://www.youtube.com/watch?v=U-Z_bZS8t3M

## Prérequis

In [0]:
! apt-get -q -y install net-tools        # ifconfig 
! apt-get -q -y install dnsutils         # nslookup
! apt-get -q -y install traceroute       # traceroute
! apt-get -q -y install python3-scapy    # scapy

Reading package lists...
Building dependency tree...
Reading state information...
net-tools is already the newest version (1.60+git20161116.90da8a0-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
dnsutils is already the newest version (1:9.11.3+dfsg-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
traceroute is already the newest version (1:2.1.0-2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
python3-scapy is already the newest version (0.23-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


## Qui suis-je ?

In [0]:
! ifconfig

eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 172.28.0.2  netmask 255.255.0.0  broadcast 172.28.255.255
        ether 02:42:ac:1c:00:02  txqueuelen 0  (Ethernet)
        RX packets 17535  bytes 9226942 (9.2 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 16004  bytes 3873901 (3.8 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 3721  bytes 1190041 (1.1 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 3721  bytes 1190041 (1.1 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0



## Où vais-je ?

In [0]:
! nslookup www.perdu.com

Server:		127.0.0.11
Address:	127.0.0.11#53

Non-authoritative answer:
Name:	www.perdu.com
Address: 208.97.177.124



## Traceroute (commande Unix)

In [0]:
! traceroute -I -4 -n www.perdu.com

traceroute to www.perdu.com (208.97.177.124), 30 hops max, 60 byte packets
 1  172.28.0.1  0.033 ms  0.007 ms  0.005 ms
 2  216.239.54.115  27.614 ms  27.639 ms  27.637 ms
 3  209.85.252.121  14.399 ms  14.398 ms  14.389 ms
 4  172.253.65.166  82.096 ms  82.097 ms  82.095 ms
 5  209.85.254.238  84.463 ms  84.620 ms  84.647 ms
 6  108.170.248.4  85.342 ms  85.286 ms  85.279 ms
 7  205.197.230.37  82.461 ms  81.015 ms  80.993 ms
 8  207.88.13.113  87.632 ms  87.729 ms  87.698 ms
 9  209.48.43.58  88.657 ms  88.220 ms  88.184 ms
10  208.113.156.4  90.053 ms  90.169 ms  89.039 ms
11  208.113.156.14  90.436 ms  90.419 ms  92.542 ms
12  208.97.177.124  87.786 ms  87.786 ms  87.787 ms


## Comment ça marche ?

Chaque paquet IP a une durée de vie initiale maximum, le champs TTL (Time To Leave), qui est décrémenté au passage de chaque routeur. Lorsque le TTL atteint 0, le routeur jette le paquet à la poubelle et émet un paquet ICMP d'erreur *Time to live exceeded* vers la source. 

* A quoi ça sert ?
* En déduire ainsi un moyen de découvrir les routeurs intermédiaires de proche en proche.

=> https://fr.wikipedia.org/wiki/Traceroute



## Version Python/Scapy

=> https://scapy.readthedocs.io/en/latest/usage.html

Privilège *root* requis.

In [0]:
from scapy.all import *

def mytraceroute(target, maxttl):
    for k in range(1, maxttl):
        x = IP(version=4, dst=target, ttl=k)/ICMP()
        y = sr1(x, verbose=0)
        if y.getlayer(ICMP).type == 11 and y.getlayer(ICMP).code == 0:    # time to leave exceeded (11)
            print(k, y.src)
        elif y.getlayer(ICMP).type == 0 and y.getlayer(ICMP).code == 0:   # echo-reply (0)
            print(k, y.src)
            break
        else:
            print(k, "* * *")


mytraceroute("www.perdu.com", 64)

1 172.28.0.1
2 216.239.54.115
3 209.85.252.121
4 172.253.65.166
5 209.85.254.238
6 108.170.248.4
7 205.197.230.37
8 207.88.13.113
9 209.48.43.58
10 208.113.156.4
11 208.113.156.14
12 208.97.177.124
